The caching algorithm in this repo is based on Online Linear Optimization. Here is a small explainer with missing details.

# Online Linear Optimization (OLO)

## Static Regret
Online Linear Optimization (OLO) is a framework where, at each round, a decision maker selects an action from a convex set before a linear loss function is revealed. The goal is to minimize the cumulative loss compared to the best fixed action in hindsight—a difference known as **static regret**.

In OLO, the **static regret** after $T$ rounds is defined as
$$
\text{Regret}_T = \sum_{t=1}^T \langle g_t, x_t \rangle - \min_{x \in K} \sum_{t=1}^T \langle g_t, x \rangle,
$$ 

where $x_t \in K$ is the decision at round $t$, $g_t$ is the gradient (or loss vector) revealed at round $t$ , and $K$ is the convex decision set.


## Online Projected Gradient Descent (OPGD)
A popular algorithm in this setting is **Online Projected Gradient Descent (OPGD)**. At each iteration, the algorithm updates the current decision by stepping in the direction of the negative gradient of the loss and then projects back onto the feasible set to maintain constraints. This approach leverages the convexity of the decision set and the linearity of losses, offering a simple yet effective method for minimizing static regret over time.


The update rule for OPGD is

$$
x_{t+1} = \Pi_K\Bigl( x_t - \eta\, g_t \Bigr),
$$ 

where $\eta > 0$ is the learning rate, and $\Pi_K(y) = \arg\min_{x \in K} \|x - y\|$   denotes the projection of $y$  onto the set $K$.

## Regret bound for OPGD
A central result in OLO is that **OPGD achieves a regret bound on the order of $\sqrt{T}$** for a well chosen $\eta$ when $K$ is bounded and convex, meaning that

$$
\text{Regret}_T = O(\sqrt{T}).
$$ 

This $\sqrt{T}$ bound is important because it implies that the **average regret per round**, $\frac{\text{Regret}_T}{T}$, tends to zero as $T$ increases, ensuring that the algorithm performs nearly as well as the best fixed decision in hindsight even against adversarial losses.

The key intuition behind this result is that, with an appropriately chosen learning rate $\eta$, the incremental loss incurred by each OPGD update can be controlled through the properties of convexity and the geometry of the projection step. By carefully balancing the step size and the accumulated errors (using a telescoping sum argument), one shows that the total deviation from the best fixed action does not exceed a term proportional to $\sqrt{T}$. This makes OPGD particularly effective in the online setting, where decisions must be made sequentially without prior knowledge of future loss functions.


## Sources OLO
- o3 mini (
write a text introducing OLO online linear optimization, keep it short introduce static regret and online projected gradient descent, explain that an important result or the essence why OLO works is that OPGD obtains sqrt T regret
)

- Hazan: http://arxiv.org/abs/1909.05207 (Introduction to Online Convex Optimization)

- Ashoks Thesis: https://www-cs.stanford.edu/people/ashokc/papers/thesis.pdf (PhD on PARAMETER-FREE ONLINE
LEARNING )

- Orabona:  http://arxiv.org/abs/1912.13213 (A Modern Introduction to Online Learning)

- convex set wiki: https://en.wikipedia.org/wiki/Convex_set

# OLO framework for caching


Fractional caching can be analyzed as a OLO problem. 

## Representing fractional caching strategy
Let $y$ be a vector where each component $y_{i}$ represents the fraction of item $i$ that is cached,  the set of feasibility for y is (for a cache of size $C$): 

$$
Y = \left\{ y \in [0,1]^N \middle| \sum_{n=1}^N y^{n} \leq C \right\},
$$ 

with $N$ the max possible different items, as there always exist a full cache strategy that outperforms a non full cache strategy all our fractional cache states will be in

$$
Y_{full} = \left\{ y \in [0,1]^N \middle| \sum_{n=1}^N y^{n} = C \right\}.
$$ 
Note that it is essential that the feasibility set is convex, for integral caching this is not the case so OLO isn't directly applicable.


## Optimal Factional Static Cache 

To understand how fractional caching is an OLO problem, you can also understand how you can frame finding the optimal fractional static cache in hindsight as a constrained convex linear optimization problem or in this case linear programming problem because 
$\mathcal{Y}$ is a linear constraint.

To pose a optimization problem we should chose our goal, loss/utility function. In the case of caching our goal is to minimize the miss rate or equivalently maximize the hit rate. We define the hit rate for a sequence of requests $l_i \in \mathbb{N}$ ($l_{t}$ is the number of the $t$ th requested item) of length $T$ for a static cache $y \in Y_{full}$ as:

$$
H_{T}(y) = \frac{1}{T}\sum_{t=1}^{T} y^{l_t} .
$$ 
to keep the $g_{t}$ the same for different $T$ later we ignore $\frac{1}{T}$ and added it later in doing this preserves the optimization problem, to make this similar to the static regret definition which uses minimization we are going to minimize the negative hit rate and use basis vectors $e^{l_{t}}$ to select component $l_{t}$ of $y$ with the inner product, so finding the optimal fractional static cache in hindsight $y_{opt}$ is framed as following optimization problem:

\begin{align*}
y_{opt} &= \argmin_{y \in Y_{full}} \left(- \sum_{t=1}^{T} \langle e^{l_{t}} , y \rangle \right) \\
        &= \argmin_{y \in Y} \left( \sum_{t=1}^{T} \langle g_{t} , y \rangle \right) 
\end{align*}
with $g_{t} = - e^{l_{t}} $, the $g_{t}$ s are the gradients in our OLO caching problem. Now formulating caching as an OLO problem is obvious. An interesting extension is to let $C$ be variable at each time step and adding $C$ to the loss function and constraining it to an interval this way the online caching strategy can also decide on how big to chose the cache. 

## OPGD for caching

Applying OPGD on our fractional caching formulation defines following online fractional caching strategy initialize with an arbitrary $y_{0}\in Y$ then:

\begin{align*}
y_{t+1} &= \Pi_Y \Bigl( y_t - \eta\, g_t \Bigr), \\
        &= \Pi_Y \Bigl( y_t + \eta\, e^{l_{t}} \Bigr) 

\end{align*}



this strategy comes with the regret guarantee of OPGD: 

\begin{align*}
O(\sqrt{T}) &= \sum_{t=1}^T \langle g_t, y_t \rangle -  \sum_{t=1}^T \langle g_t, y_{OPT} \rangle,  \Leftrightarrow \\
\frac{O(\sqrt{T})}{T} &= \frac{1}{T}\sum_{t=1}^T \langle g_t, y_t \rangle -  \frac{1}{T} \sum_{t=1}^T \langle g_t, y_{OPT} \rangle, \Leftrightarrow \\
\frac{O(\sqrt{T})}{T} &= - H_{T}(y_{t}) +  H_{T}(y_{OPT}), \Leftrightarrow \\
\frac{O(\sqrt{T})}{T} &=  H_{T}(y_{OPT})- H_{T}(y_{t})  ,  \\
\end{align*}

this means as $T\rightarrow \infty$ the hit rate of the online strategy converges to the hit rate of the static optimal in hindsight.


## Sources OLO framework for caching
- o3 mini (explain the OLO framework for fractional caching)
- paper Paschos: http://arxiv.org/abs/1904.09849 (introduces OGA $\approx$ OPGD for caching)
- wiki linear programming: https://en.wikipedia.org/wiki/Linear_programming


# fast OPGD for fractional caching 

Throughput and latency are important considerations for caching algorithms. A caching algorithm can be impractical when incoming request arrive faster then the throughput so that the waiting time for handling each request grows unboundedly or when the latency introduced by computing and executing the dynamical cache takes longer then having no cache.   

## steps of OPGD for fractional caching

OPGD for fractional caching consists out of following steps: 

- step -1: receive $l_{t}$ (part of the definition of the caching problem, we don't touch this)  <br>
- step 0: get the fraction of cached $l_{t}$ physically (performance depends on how this is stored)  <br>
- step 1: the gradient update $+ \eta e^{l_{t}}$ (can be achieved in $O(1)$ time, so we don't touch this)  <br>
- step 2: projecting on $Y$ (changes at best all nonzero components of $y_{t}$ but is very symmetric) <br>
- step 3: updating the cache physically from $y_{t}$ to $y_{t+1}$(adds fraction to $l_{t}$, and removes from all others)



## OPGD: making the projection fast

One way to think about this projection is to consider the KKT conditions for the corresponding optimization problem but we think this is overkill.

We start by assuming that by initializing the caching strategy in $y_{0} \in Y_{full}$ all projections in later iterations on $Y$ are equal to the projections on $Y_{full}$ or we get a better online caching strategy. (we haven't figured out a simple formal argument for this but to us it visually obvious, $Y$ for $C>1$ is the intersection of a box and a half plane, a n-dimensional box with a cut corner) 

Observe that $Y_{full}$ is an intersection of planes and  box constraints:

$$
Y_{full} = [0,\infty[^{n}  \cap  ] \infty,1 ]^{n} \cap  \left\{y \in \mathbb{R}^n \middle| \sum_{n=1}^{N} y^{n} = C \right\} .
$$ 

Consider the method of alternating projections for these $3$ convex sets, projection on $[0,1]^{n}$ is clipping all the components between $0$ and $1$ and projecting on the plane is subtracting the normal vector with a magnitude the distance in this case it is the $1$ vector (normalized) with magnitude after step 1: 

$$
m_{t} = \sum_{n=1}^{N} y_{t}^{n} + \eta   - C = \eta. 
$$ 

Just vanilla alternating methods of projections would converge geometrically fast but in this case we can improve it to exact in finite amount of steps through following observations:


- after projecting first on $] \infty,1 ]^{n}$ all following projections stay in $] \infty,1 ]^{n}$ meaning that we only have to project once on $] \infty,1 ]^{n}$, this property follows by the fact that the other projections only reduce the components or maximally brings a component to $0$  

- in the first projection on $] \infty,1 ]^{n}$, you only have to check the component that is updated in step 1 ($O(1)$ time )

- after you ignore projecting on $] \infty,1 ]^{n}$ any components that are projected to $0$ should be $0$, this follows from the fact that the minimum of a component in $Y_{full}$ is $0$ and after being projected on $0$ it can only be reduced or maximally brought to $0$ so we found a matching lower and upper bound giving the equality for $0$ 

- maximum $N$ components can be $0$ because of step 1 in each iteration at most one component transitions from zero to non-zero. Consequently, amortized over all iterations, the algorithm can set at most one component to zero on average.

Assuming, for contradiction, that on average $1 + \varepsilon$  components are set to zero per iteration, the net change in the number of zero components would be at least $\varepsilon$   per step. Over $1 + \frac{N}{\varepsilon}$ iterations, this would result in an increase of more than $N$ zero components. However, this exceeds the initial maximum $N$ zero components, leading to a contradiction. Thus, the average number of components set to zero per iteration cannot exceed one.

- you can avoid redoing the same $0$ component projections by doing projections on the plane in the subspace where the $0$ components are $0$, here the normal vector is also the $1$ vector but has $0$ at the $0$ components, you can still subtract by the full $1$ vector if you treat all negative components as $0$ later 

- the projections on the plane (subtractions of the normal vector) can be implemented lazily, i.e. never execute the subtraction until you need a component so at the time of the projection you only have to update how much you should subtract by the normal vector later, a more geometric way of thinking is that instead of updating all the components of a point in n-dimensions, you change the coordinate system by moving the origin in a straight line of the normal vector and you keep track how far it went ($O(1)$ time at projection, $O(1)$ time per subtraction when accessing a component) 

Note that the lazy subtraction influences possibly all aspects of the algorithm.

- detection of zero components can be done by maintaining on ordered list of the component of $y$ as in all steps up until now maximum $O(1)$ components change maintance cost $O(log(N))$ note that lazy subtraction doesn't effect the order of the components  ($O(log(N))$ look up time )


So all these observations combined means that we on average only have to do $4$ projections to get the exact result and it at most cost us $O(log(N))$ time.

## Limitation of fractional caching

Implementing Steps 0 and 3 efficiently simultaneously seems difficult. For step 0, the fractions of an item should be stored together to enable quick access. Conversely, Step 3 requires storing different fractions of items together to allow for fast deletion.


# fast OPGD for integral caching 

By using randomized rounding schemes fractional caching algorithms can be made integral. So now we have to keep track of $y_{t}$ and the current integral cache $X_{t}$ with $y_{t} = E[X_{t}]$ this assures that: 

\begin{align*}
\text{Regret}_T &= \sum_{t=1}^T \langle g_t, y_t \rangle - \min_{y \in Y} \sum_{t=1}^T \langle g_t, y \rangle, \\
&= \sum_{t=1}^T \langle g_t, E[X_{t}] \rangle- \min_{y \in Y} \sum_{t=1}^T \langle g_t, y \rangle \\
&= E\left[\sum_{t=1}^T \langle g_t, X_{t} \rangle\right]- \min_{y \in Y} \sum_{t=1}^T \langle g_t, y \rangle \\
\end{align*}





## Sources fast OPGD for caching

- wiki projection on convex set: https://en.wikipedia.org/wiki/Projections_onto_convex_sets
- paper Duchi: http://portal.acm.org/citation.cfm?doid=1390156.1390191 (also uses lazy updates to project on the simplex)
- paper Carra: http://arxiv.org/abs/2405.01263 (introduces efficient implementations for fractional and integral caching)
- paper Salem: http://arxiv.org/abs/2101.12588 (departs from fractional caching to integral caching my a rounding sheme)


# Quantized Online Caching Descent (qOCD)



